# Precalculating AOI information
The site will show precalculated information for: WWF ecorregions, GADM admin 1 and 2 and WDPA. The minimum size of the features to be able to provide biodiversity data must be over 1000km^2. 
- Ecorregions polygon used ([arcgis online item](https://eowilson.maps.arcgis.com/home/item.html?id=be0308e81950409b919940bd03f2cae6))
- GADM polygons, level 0 and 1 (downloaded from [here](https://gadm.org/download_world.html))
- WDPA raster (june version, [provided by MOL](https://eowilson.maps.arcgis.com/home/content.html?user=MoL_HEdata&view=table&sortOrder=desc&sortField=modified&folder=1c7e82b763e54da29644eef4e048b2df#content))

**NOTE**: the polygons used to extract the information are not the same geometries shown on the app. In the app the geometries have been generalised to increase performance.


## Information to be gathered
- `population_sum`
- `land_cover_majority`
- `climate_regime_majority`
- `urban_percent`
- `rainfed_percent`
- `irrigated_percent`
- `rangeland_percent`
- `number_of_mammals`
- `number_of_amphibians`
- `number_of_birds`
- `number_of_reptiles`
- `percentage_protected`
- `mammals_list`
- `amphibians_list`
- `birds_list`
- `reptiles_list`

### Ecoregions

#### Population

In [ ]:
arcpy.sa.ZonalStatisticsAsTable("Ecoregions2017_20210825", 
                                "eco_id_int", 
                                "population2020.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\ecoregion_population_sum", 
                                "DATA", "SUM", "CURRENT_SLICE", 90)

In [ ]:
arcpy.management.JoinField("Ecoregions2017_20210825", "eco_id_int", "ecoregion_population_sum", "eco_id_int", "SUM")

In [ ]:
arcpy.management.AlterField("Ecoregions2017_20210825", "SUM", 
                            "population_sum", "population_sum", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

#### climate regime and land cover

In [ ]:
arcpy.sa.ZonalStatisticsAsTable("Ecoregions2017_20210825", 
                                "eco_id_int", "ELU.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\ecoregion_elu_majority", 
                                "DATA", "MAJORITY", "CURRENT_SLICE", [90])

In [ ]:
arcpy.management.JoinField("Ecoregions2017_20210825", "eco_id_int", "ecoregion_elu_majority", "eco_id_int", "MAJORITY")

In [ ]:
arcpy.management.AlterField("Ecoregions2017_20210825", "MAJORITY", "majority_lc_cr", "majority land cover climate regime", 
                            "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")

In [ ]:
arcpy.management.JoinField("Ecoregions2017_20210825", 
                           "majority_lc_cr", 
                           r"C:\Users\Viz1\Documents\ArcGIS\Projects\mammal_data_cube\elu_categories_lookup.csv", 
                           "elu_code", "lc_type;cr_type")

In [ ]:
arcpy.management.AlterField("Ecoregions2017_20210825", "lc_type", "land_cover_majority", "land_cover_majority", "TEXT", 8000, "NULLABLE", "DO_NOT_CLEAR")

In [ ]:
arcpy.management.AlterField("Ecoregions2017_20210825", "cr_type", "climate_regime_majority", "climate_regime_majority", "TEXT", 8000, "NULLABLE", "DO_NOT_CLEAR")

#### Protected percentage


In [4]:
arcpy.sa.ZonalStatisticsAsTable("Ecoregions2017_20210825_PolygonToRaster", "Value", "wdpa_oecm_zeros.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\wdpa_ecoregions_mean", 
                                "DATA", "MEAN", "CURRENT_SLICE", 90)

<geoprocessing server result object object at 0x0000029825526E68>

In [5]:
arcpy.management.JoinField("Ecoregions2017_20210825", "eco_id_int", 
                           "wdpa_ecoregions_mean", "Value", "MEAN")

<Result 'Ecoregions2017_20210825'>

In [ ]:
arcpy.management.AlterField("Ecoregions2017_20210825", "MEAN", "percentage_protected", "percentage_protected", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

#### Land encroachment
look up table:
- 1: irrigated
- 2: rainfed
- 3: rangeland
- 4: urban

In [ ]:

arcpy.conversion.PolygonToRaster("Ecoregions2017_20210825", "eco_id_int", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\Ecoregions2017_20210825_PolygonToRaster", "CELL_CENTER", "NONE", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\land_encroachment_Subset.crf")

In [ ]:
arcpy.sa.ZonalStatisticsAsTable("Ecoregions2017_20210825", "eco_id_int", "land_encroachment.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\land_encroachment_ecoregions_all", 
                                "DATA", "ALL", "ALL_SLICES", [90])

In [ ]:
# join field from the raster
arcpy.management.JoinField("land_encroachment_ecoregions_all", "eco_id_int", 
                           "Ecoregions2017_20210825_PolygonToRaster", "Value", "Count")



In [21]:
# create new field and calculate the percent within land_encroachment_ecoregions_all,
arcpy.management.CalculateField("land_encroachment_ecoregions_all", "percent_encroachment", 
                                "round((!COUNT! / !Count_1!)*100,2)", "PYTHON3", '', "TEXT")

<Result 'land_encroachment_ecoregions_all'>

In [32]:
# select the rows for each kind of encroachment. join the field and change the name
# 1 irrigated
val = 1
slice_name = "irrigated"
arcpy.analysis.TableSelect("land_encroachment_ecoregions_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_ecoregions", f"SliceNumber = {val}")
arcpy.management.JoinField(f"Ecoregions2017_20210825", "eco_id_int", f"land_{slice_name}_ecoregions", "eco_id_int", "percent_encroachment")
arcpy.management.AlterField("Ecoregions2017_20210825", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'Ecoregions2017_20210825'>

In [33]:
val = 2
slice_name = "rainfed"
arcpy.analysis.TableSelect("land_encroachment_ecoregions_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_ecoregions", f"SliceNumber = {val}")
arcpy.management.JoinField(f"Ecoregions2017_20210825", "eco_id_int", f"land_{slice_name}_ecoregions", "eco_id_int", "percent_encroachment")
arcpy.management.AlterField("Ecoregions2017_20210825", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'Ecoregions2017_20210825'>

In [34]:
# select the rows for each kind of encroachment. join the field and change the name

val = 3
slice_name = "rangeland"
arcpy.analysis.TableSelect("land_encroachment_ecoregions_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_ecoregions", f"SliceNumber = {val}")
arcpy.management.JoinField(f"Ecoregions2017_20210825", "eco_id_int", f"land_{slice_name}_ecoregions", "eco_id_int", "percent_encroachment")
arcpy.management.AlterField("Ecoregions2017_20210825", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'Ecoregions2017_20210825'>

In [35]:
# select the rows for each kind of encroachment. join the field and change the name

val = 4
slice_name = "urban"
arcpy.analysis.TableSelect("land_encroachment_ecoregions_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_ecoregions", f"SliceNumber = {val}")
arcpy.management.JoinField(f"Ecoregions2017_20210825", "eco_id_int", f"land_{slice_name}_ecoregions", "eco_id_int", "percent_encroachment")
arcpy.management.AlterField("Ecoregions2017_20210825", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'Ecoregions2017_20210825'>

## GADM
Using non generalised polygons to extract the information
- population
- percentage protected (June version, MOL data)
- ecosystem majority

In [ ]:
arcpy.management.Merge("gadm_level1;gadm_level0", 
                       r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\gadm_level0_and_1_Merge", 
                       'GID_0 "GID_0" true true false 2147483647 Text 0 0,First,#,gadm_level1,GID_0,0,2147483647,gadm_level0,GID_0,0,2147483647;NAME_0 "NAME_0" true true false 2147483647 Text 0 0,First,#,gadm_level1,NAME_0,0,2147483647,gadm_level0,NAME_0,0,2147483647;GID_1 "GID_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,GID_1,0,2147483647;NAME_1 "NAME_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,NAME_1,0,2147483647;VARNAME_1 "VARNAME_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,VARNAME_1,0,2147483647;NL_NAME_1 "NL_NAME_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,NL_NAME_1,0,2147483647;TYPE_1 "TYPE_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,TYPE_1,0,2147483647;ENGTYPE_1 "ENGTYPE_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,ENGTYPE_1,0,2147483647;CC_1 "CC_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,CC_1,0,2147483647;HASC_1 "HASC_1" true true false 2147483647 Text 0 0,First,#,gadm_level1,HASC_1,0,2147483647;geom_Length "geom_Length" false true true 8 Double 0 0,First,#,gadm_level1,geom_Length,-1,-1,gadm_level0,geom_Length,-1,-1;geom_Area "geom_Area" false true true 8 Double 0 0,First,#,gadm_level1,geom_Area,-1,-1,gadm_level0,geom_Area,-1,-1;country_size "country_size" true true false 4 Long 0 0,First,#,gadm_level1,country_size,-1,-1,gadm_level0,country_size,-1,-1;country_zoom "country_zoom" true true false 4 Long 0 0,First,#,gadm_level0,country_zoom,-1,-1', "NO_SOURCE_INFO")

In [7]:
arcpy.sa.ZonalStatisticsAsTable("gadm_level0_and_1_Merge", 
                                "GID_0_1", 
                                "population2020.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\gadm_population_sum", 
                                "DATA", "SUM", "CURRENT_SLICE", 90)

<geoprocessing server result object object at 0x0000029825526EB8>

In [8]:
arcpy.management.JoinField("gadm_level0_and_1_Merge", 
                           "GID_0_1", 
                           "gadm_population_sum", 
                           "GID_0_1", "SUM")

<Result 'gadm_level0_and_1_Merge'>

In [9]:
arcpy.management.AlterField("gadm_level0_and_1_Merge", "SUM", 
                            "population_sum", "population_sum", 
                            "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

#### climate regime and land cover

In [10]:
arcpy.sa.ZonalStatisticsAsTable("gadm_level0_and_1_Merge", 
                                "GID_0_1", "ELU.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\gadm_elu_majority", 
                                "DATA", "MAJORITY", "CURRENT_SLICE", [90])

<geoprocessing server result object object at 0x0000029825526FA8>

In [11]:
arcpy.management.JoinField("gadm_level0_and_1_Merge", 
                           "GID_0_1", "gadm_elu_majority", "GID_0_1", "MAJORITY")

<Result 'gadm_level0_and_1_Merge'>

In [12]:
arcpy.management.AlterField("gadm_level0_and_1_Merge", "MAJORITY", "majority_lc_cr", "majority land cover climate regime", 
                            "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

In [13]:
arcpy.management.JoinField("gadm_level0_and_1_Merge", 
                           "majority_lc_cr", 
                           r"C:\Users\Viz1\Documents\ArcGIS\Projects\mammal_data_cube\elu_categories_lookup.csv", 
                           "elu_code", "lc_type;cr_type")

<Result 'gadm_level0_and_1_Merge'>

In [14]:
arcpy.management.AlterField("gadm_level0_and_1_Merge", "lc_type", "land_cover_majority", "land_cover_majority", "TEXT", 8000, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

In [15]:
arcpy.management.AlterField("gadm_level0_and_1_Merge", "cr_type", "climate_regime_majority", "climate_regime_majority", "TEXT", 8000, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

#### Protected percentage


In [18]:
arcpy.sa.ZonalStatisticsAsTable("gadm_level0_and_1_Merge", "GID_0_1", "wdpa_oecm_zeros.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\wdpa_gadm_mean", "DATA", "MEAN", "CURRENT_SLICE", 90)

<geoprocessing server result object object at 0x0000029802FCA530>

In [19]:
arcpy.management.JoinField("gadm_level0_and_1_Merge", "GID_0_1", 
                           "wdpa_gadm_mean", "GID_0_1", "MEAN")

<Result 'gadm_level0_and_1_Merge'>

In [20]:
arcpy.management.AlterField("gadm_level0_and_1_Merge", "MEAN", "percentage_protected", "percentage_protected", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

<Result 'gadm_level0_and_1_Merge'>

#### land encroachment
`@todo`

In [ ]:
arcpy.sa.ZonalStatisticsAsTable("gadm_level0_and_1_Merge", "GID_0_1", "land_encroachment.crf", 
                                r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\land_encroachment_gadm_all", 
                                "DATA", "ALL", "ALL_SLICES", 90)

In [ ]:
arcpy.conversion.PolygonToRaster("gadm_level0_and_1_Merge", "GID_0_1", 
                                 r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\gadm_level_0_and_1_PolygonToRaster", "CELL_CENTER", "NONE", 
                                 r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\land_encroachment_Subset.crf")

In [ ]:
# join field from the raster
arcpy.management.JoinField("land_encroachment_ecoregions_all", "eco_id_int", 
                           "Ecoregions2017_20210825_PolygonToRaster", "Value", "Count")



In [21]:
# create new field and calculate the percent within land_encroachment_ecoregions_all,
arcpy.management.CalculateField("land_encroachment_ecoregions_all", "percent_encroachment", 
                                "round((!COUNT! / !Count_1!)*100,2)", "PYTHON3", '', "TEXT")

<Result 'land_encroachment_ecoregions_all'>

In [36]:
val = 1
slice_name = "irrigated"
arcpy.analysis.TableSelect("land_encroachment_gadm_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_gadm01", f"SliceNumber = {val}")
arcpy.management.JoinField(f"gadm_level0_and_1_Merge", "GID_0_1", f"land_{slice_name}_gadm01", "GID_0_1", "percent_encroachment")
arcpy.management.AlterField("gadm_level0_and_1_Merge", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000728: Field percent_encroachment does not exist within table
Failed to execute (AlterField).


In [ ]:
val = 2
slice_name = "rainfed"
arcpy.analysis.TableSelect("land_encroachment_gadm_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_gadm01", f"SliceNumber = {val}")
arcpy.management.JoinField(f"gadm_level0_and_1_Merge", "GID_0_1", f"land_{slice_name}_gadm01", "GID_0_1", "percent_encroachment")
arcpy.management.AlterField("gadm_level0_and_1_Merge", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

In [ ]:
val = 3
slice_name = "rangeland"
arcpy.analysis.TableSelect("land_encroachment_gadm_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_gadm01", f"SliceNumber = {val}")
arcpy.management.JoinField(f"gadm_level0_and_1_Merge", "GID_0_1", f"land_{slice_name}_gadm01", "GID_0_1", "percent_encroachment")
arcpy.management.AlterField("gadm_level0_and_1_Merge", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

In [ ]:
val = 4
slice_name = "urban"
arcpy.analysis.TableSelect("land_encroachment_gadm_all", f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/land_{slice_name}_gadm01", f"SliceNumber = {val}")
arcpy.management.JoinField(f"gadm_level0_and_1_Merge", "GID_0_1", f"land_{slice_name}_gadm01", "GID_0_1", "percent_encroachment")
arcpy.management.AlterField("gadm_level0_and_1_Merge", "percent_encroachment", f"percent_{slice_name}", f"percent_{slice_name}", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")

### Creating service with generalised feature classes

In the country level give a size category to each country. Then use that field in the admin1 level to create the different feature collections per zoom level. 
Once the level 1 have been divided, create a new field in the country that is one more value than the previously calculated.

- Living atlas [service](https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/World_Administrative_Divisions/FeatureServer) with level 1 admin borders
- Generalised [service](https://services9.arcgis.com/IkktFdUAcY3WrH25/arcgis/rest/services/gadm_generalised/FeatureServer) with the country borders used in the National Report Cards (missing Fiji and Kiribati)
- Fiji and Kiribati [service](https://services9.arcgis.com/IkktFdUAcY3WrH25/arcgis/rest/services/KIR_FIJ_proj4716/FeatureServer)

In [ ]:
arcpy.analysis.Select("World_Administrative_Divisions", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\World_Administrative_Divisio", "DISPUTED <= 1")

In [ ]:
with arcpy.EnvManager(outputCoordinateSystem="GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]"):
    arcpy.management.Merge("KIR_FIJ_proj4716;LA_GADM_clean_dateline", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\generalised_nrc", 'GID_0 "GID_0" true true false 8000 Text 0 0,First,#,KIR_FIJ_proj4716,GID_0,0,8000,LA_GADM_clean_dateline,GID_0,0,8000;NAME_0 "NAME_0" true true false 8000 Text 0 0,First,#,KIR_FIJ_proj4716,NAME_0,0,8000,LA_GADM_clean_dateline,NAME_0,0,8000;Shape__Area "Shape__Area" false true false 0 Double 0 0,First,#,KIR_FIJ_proj4716,Shape__Area,-1,-1,LA_GADM_clean_dateline,Shape__Area,-1,-1;Shape__Length "Shape__Length" false true false 0 Double 0 0,First,#,KIR_FIJ_proj4716,Shape__Length,-1,-1,LA_GADM_clean_dateline,Shape__Length,-1,-1', "NO_SOURCE_INFO")

In [ ]:
arcpy.management.CalculateField("generalised_nrc", "country_size", "country_size(!Shape_Area!)", "PYTHON3", r"def country_size(poly_area):
    if poly_area <= 10:
        a = 5
    if poly_area>10 and poly_area<=30:
        a = 4
    if poly_area>30 and poly_area<=100:
        a =3
    if poly_area>100 and poly_area<=952:
        a =2
    if poly_area>952 and poly_area<=6053:
         a =1
    return a", "LONG")

In [ ]:
arcpy.management.JoinField("generalised_nrc", "GID_0", "countrycode.csv", "iso3", "iso2")

In [ ]:
arcpy.management.JoinField("World_Administrative_Divisio", "ISO_CC", "generalised_nrc", "iso2", "GID_0;country_size")

In [2]:
for zl in [1,2,3,4,5]:
    arcpy.analysis.Select("World_Administrative_Divisio", 
                          f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/World_Administrative_Divisio_zoom_{zl}", f"country_size = {zl}")

In [3]:
for zl in [0,1,2,3,4]:
    arcpy.analysis.Select("generalised_nrc", 
                          f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/generalised_nrc_zoom{zl}", f"country_zoom = {zl}")

#### Creating a service per zoom level. 

In [ ]:
arcpy.management.Merge(r"zoom4\World_Administrative_Divisio_zoom_4;zoom4\generalised_nrc_zoom4;zoom4\World_Administrative_Divisio_zoom_3;zoom4\World_Administrative_Divisio_zoom_2;zoom4\World_Administrative_Divisio_zoom_1", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\zoom4_Merge", r'NAME "Name" true true false 44 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,NAME,0,44,zoom4\World_Administrative_Divisio_zoom_3,NAME,0,44,zoom4\World_Administrative_Divisio_zoom_2,NAME,0,44,zoom4\World_Administrative_Divisio_zoom_1,NAME,0,44;COUNTRY "Country" true true false 50 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,COUNTRY,0,50,zoom4\World_Administrative_Divisio_zoom_3,COUNTRY,0,50,zoom4\World_Administrative_Divisio_zoom_2,COUNTRY,0,50,zoom4\World_Administrative_Divisio_zoom_1,COUNTRY,0,50;ISO_CODE "ISO Code" true true false 6 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,ISO_CODE,0,6,zoom4\World_Administrative_Divisio_zoom_3,ISO_CODE,0,6,zoom4\World_Administrative_Divisio_zoom_2,ISO_CODE,0,6,zoom4\World_Administrative_Divisio_zoom_1,ISO_CODE,0,6;ISO_CC "ISO Country" true true false 2 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,ISO_CC,0,2,zoom4\World_Administrative_Divisio_zoom_3,ISO_CC,0,2,zoom4\World_Administrative_Divisio_zoom_2,ISO_CC,0,2,zoom4\World_Administrative_Divisio_zoom_1,ISO_CC,0,2;ISO_SUB "ISO Admin" true true false 5 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,ISO_SUB,0,5,zoom4\World_Administrative_Divisio_zoom_3,ISO_SUB,0,5,zoom4\World_Administrative_Divisio_zoom_2,ISO_SUB,0,5,zoom4\World_Administrative_Divisio_zoom_1,ISO_SUB,0,5;ADMINTYPE "Admin Type" true true false 50 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,ADMINTYPE,0,50,zoom4\World_Administrative_Divisio_zoom_3,ADMINTYPE,0,50,zoom4\World_Administrative_Divisio_zoom_2,ADMINTYPE,0,50,zoom4\World_Administrative_Divisio_zoom_1,ADMINTYPE,0,50;DISPUTED "Disputed" true true false 4 Long 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,DISPUTED,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,DISPUTED,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,DISPUTED,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,DISPUTED,-1,-1;NOTES "Notes" true true false 254 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,NOTES,0,254,zoom4\World_Administrative_Divisio_zoom_3,NOTES,0,254,zoom4\World_Administrative_Divisio_zoom_2,NOTES,0,254,zoom4\World_Administrative_Divisio_zoom_1,NOTES,0,254;AUTONOMOUS "Autonomous" true true false 4 Long 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,AUTONOMOUS,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,AUTONOMOUS,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,AUTONOMOUS,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,AUTONOMOUS,-1,-1;COUNTRYAFF "Affiliation" true true false 100 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,COUNTRYAFF,0,100,zoom4\World_Administrative_Divisio_zoom_3,COUNTRYAFF,0,100,zoom4\World_Administrative_Divisio_zoom_2,COUNTRYAFF,0,100,zoom4\World_Administrative_Divisio_zoom_1,COUNTRYAFF,0,100;CONTINENT "Continent" true true false 13 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,CONTINENT,0,13,zoom4\World_Administrative_Divisio_zoom_3,CONTINENT,0,13,zoom4\World_Administrative_Divisio_zoom_2,CONTINENT,0,13,zoom4\World_Administrative_Divisio_zoom_1,CONTINENT,0,13;LAND_TYPE "Land_Type" true true false 20 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,LAND_TYPE,0,20,zoom4\World_Administrative_Divisio_zoom_3,LAND_TYPE,0,20,zoom4\World_Administrative_Divisio_zoom_2,LAND_TYPE,0,20,zoom4\World_Administrative_Divisio_zoom_1,LAND_TYPE,0,20;LAND_RANK "Land_Rank" true true false 4 Long 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,LAND_RANK,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,LAND_RANK,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,LAND_RANK,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,LAND_RANK,-1,-1;GID_0 "GID_0" true true false 8000 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,GID_0,0,8000,zoom4\generalised_nrc_zoom4,GID_0,0,8000,zoom4\World_Administrative_Divisio_zoom_3,GID_0,0,8000,zoom4\World_Administrative_Divisio_zoom_2,GID_0,0,8000,zoom4\World_Administrative_Divisio_zoom_1,GID_0,0,8000;country_size "country_size" true true false 4 Long 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,country_size,-1,-1,zoom4\generalised_nrc_zoom4,country_size,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,country_size,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,country_size,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,country_size,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,Shape_Length,-1,-1,zoom4\generalised_nrc_zoom4,Shape_Length,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,Shape_Length,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,Shape_Length,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,Shape_Area,-1,-1,zoom4\generalised_nrc_zoom4,Shape_Area,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,Shape_Area,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,Shape_Area,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,Shape_Area,-1,-1;NAME_0 "NAME_0" true true false 8000 Text 0 0,First,#,zoom4\generalised_nrc_zoom4,NAME_0,0,8000;country_zoom "country_zoom" true true false 4 Long 0 0,First,#,zoom4\generalised_nrc_zoom4,country_zoom,-1,-1;iso2 "iso2" true true false 8000 Text 0 0,First,#,zoom4\generalised_nrc_zoom4,iso2,0,8000', "NO_SOURCE_INFO")
arcpy.management.Merge(r"zoom3\World_Administrative_Divisio_zoom_1;zoom3\World_Administrative_Divisio_zoom_2;zoom3\generalised_nrc_zoom3;zoom3\World_Administrative_Divisio_zoom_3;zoom3\generalised_nrc_zoom4", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\zoom3_Merge", r'NAME "Name" true true false 44 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,NAME,0,44,zoom3\World_Administrative_Divisio_zoom_2,NAME,0,44,zoom3\World_Administrative_Divisio_zoom_3,NAME,0,44;COUNTRY "Country" true true false 50 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,COUNTRY,0,50,zoom3\World_Administrative_Divisio_zoom_2,COUNTRY,0,50,zoom3\World_Administrative_Divisio_zoom_3,COUNTRY,0,50;ISO_CODE "ISO Code" true true false 6 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,ISO_CODE,0,6,zoom3\World_Administrative_Divisio_zoom_2,ISO_CODE,0,6,zoom3\World_Administrative_Divisio_zoom_3,ISO_CODE,0,6;ISO_CC "ISO Country" true true false 2 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,ISO_CC,0,2,zoom3\World_Administrative_Divisio_zoom_2,ISO_CC,0,2,zoom3\World_Administrative_Divisio_zoom_3,ISO_CC,0,2;ISO_SUB "ISO Admin" true true false 5 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,ISO_SUB,0,5,zoom3\World_Administrative_Divisio_zoom_2,ISO_SUB,0,5,zoom3\World_Administrative_Divisio_zoom_3,ISO_SUB,0,5;ADMINTYPE "Admin Type" true true false 50 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,ADMINTYPE,0,50,zoom3\World_Administrative_Divisio_zoom_2,ADMINTYPE,0,50,zoom3\World_Administrative_Divisio_zoom_3,ADMINTYPE,0,50;DISPUTED "Disputed" true true false 4 Long 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,DISPUTED,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,DISPUTED,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,DISPUTED,-1,-1;NOTES "Notes" true true false 254 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,NOTES,0,254,zoom3\World_Administrative_Divisio_zoom_2,NOTES,0,254,zoom3\World_Administrative_Divisio_zoom_3,NOTES,0,254;AUTONOMOUS "Autonomous" true true false 4 Long 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,AUTONOMOUS,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,AUTONOMOUS,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,AUTONOMOUS,-1,-1;COUNTRYAFF "Affiliation" true true false 100 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,COUNTRYAFF,0,100,zoom3\World_Administrative_Divisio_zoom_2,COUNTRYAFF,0,100,zoom3\World_Administrative_Divisio_zoom_3,COUNTRYAFF,0,100;CONTINENT "Continent" true true false 13 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,CONTINENT,0,13,zoom3\World_Administrative_Divisio_zoom_2,CONTINENT,0,13,zoom3\World_Administrative_Divisio_zoom_3,CONTINENT,0,13;LAND_TYPE "Land_Type" true true false 20 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,LAND_TYPE,0,20,zoom3\World_Administrative_Divisio_zoom_2,LAND_TYPE,0,20,zoom3\World_Administrative_Divisio_zoom_3,LAND_TYPE,0,20;LAND_RANK "Land_Rank" true true false 4 Long 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,LAND_RANK,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,LAND_RANK,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,LAND_RANK,-1,-1;GID_0 "GID_0" true true false 8000 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,GID_0,0,8000,zoom3\World_Administrative_Divisio_zoom_2,GID_0,0,8000,zoom3\generalised_nrc_zoom3,GID_0,0,8000,zoom3\World_Administrative_Divisio_zoom_3,GID_0,0,8000,zoom3\generalised_nrc_zoom4,GID_0,0,8000;country_size "country_size" true true false 4 Long 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,country_size,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,country_size,-1,-1,zoom3\generalised_nrc_zoom3,country_size,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,country_size,-1,-1,zoom3\generalised_nrc_zoom4,country_size,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,Shape_Length,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,Shape_Length,-1,-1,zoom3\generalised_nrc_zoom3,Shape_Length,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,Shape_Length,-1,-1,zoom3\generalised_nrc_zoom4,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,Shape_Area,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,Shape_Area,-1,-1,zoom3\generalised_nrc_zoom3,Shape_Area,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,Shape_Area,-1,-1,zoom3\generalised_nrc_zoom4,Shape_Area,-1,-1;NAME_0 "NAME_0" true true false 8000 Text 0 0,First,#,zoom3\generalised_nrc_zoom3,NAME_0,0,8000,zoom3\generalised_nrc_zoom4,NAME_0,0,8000;country_zoom "country_zoom" true true false 4 Long 0 0,First,#,zoom3\generalised_nrc_zoom3,country_zoom,-1,-1,zoom3\generalised_nrc_zoom4,country_zoom,-1,-1;iso2 "iso2" true true false 8000 Text 0 0,First,#,zoom3\generalised_nrc_zoom3,iso2,0,8000,zoom3\generalised_nrc_zoom4,iso2,0,8000', "NO_SOURCE_INFO")
arcpy.management.Merge(r"zoom2\World_Administrative_Divisio_zoom_1;zoom2\generalised_nrc_zoom2;zoom2\World_Administrative_Divisio_zoom_2;zoom2\generalised_nrc_zoom3;zoom2\generalised_nrc_zoom4", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\zoom2_Merge", r'NAME "Name" true true false 44 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,NAME,0,44,zoom2\World_Administrative_Divisio_zoom_2,NAME,0,44;COUNTRY "Country" true true false 50 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,COUNTRY,0,50,zoom2\World_Administrative_Divisio_zoom_2,COUNTRY,0,50;ISO_CODE "ISO Code" true true false 6 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,ISO_CODE,0,6,zoom2\World_Administrative_Divisio_zoom_2,ISO_CODE,0,6;ISO_CC "ISO Country" true true false 2 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,ISO_CC,0,2,zoom2\World_Administrative_Divisio_zoom_2,ISO_CC,0,2;ISO_SUB "ISO Admin" true true false 5 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,ISO_SUB,0,5,zoom2\World_Administrative_Divisio_zoom_2,ISO_SUB,0,5;ADMINTYPE "Admin Type" true true false 50 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,ADMINTYPE,0,50,zoom2\World_Administrative_Divisio_zoom_2,ADMINTYPE,0,50;DISPUTED "Disputed" true true false 4 Long 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,DISPUTED,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,DISPUTED,-1,-1;NOTES "Notes" true true false 254 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,NOTES,0,254,zoom2\World_Administrative_Divisio_zoom_2,NOTES,0,254;AUTONOMOUS "Autonomous" true true false 4 Long 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,AUTONOMOUS,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,AUTONOMOUS,-1,-1;COUNTRYAFF "Affiliation" true true false 100 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,COUNTRYAFF,0,100,zoom2\World_Administrative_Divisio_zoom_2,COUNTRYAFF,0,100;CONTINENT "Continent" true true false 13 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,CONTINENT,0,13,zoom2\World_Administrative_Divisio_zoom_2,CONTINENT,0,13;LAND_TYPE "Land_Type" true true false 20 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,LAND_TYPE,0,20,zoom2\World_Administrative_Divisio_zoom_2,LAND_TYPE,0,20;LAND_RANK "Land_Rank" true true false 4 Long 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,LAND_RANK,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,LAND_RANK,-1,-1;GID_0 "GID_0" true true false 8000 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,GID_0,0,8000,zoom2\generalised_nrc_zoom2,GID_0,0,8000,zoom2\World_Administrative_Divisio_zoom_2,GID_0,0,8000,zoom2\generalised_nrc_zoom3,GID_0,0,8000,zoom2\generalised_nrc_zoom4,GID_0,0,8000;country_size "country_size" true true false 4 Long 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,country_size,-1,-1,zoom2\generalised_nrc_zoom2,country_size,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,country_size,-1,-1,zoom2\generalised_nrc_zoom3,country_size,-1,-1,zoom2\generalised_nrc_zoom4,country_size,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,Shape_Length,-1,-1,zoom2\generalised_nrc_zoom2,Shape_Length,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,Shape_Length,-1,-1,zoom2\generalised_nrc_zoom3,Shape_Length,-1,-1,zoom2\generalised_nrc_zoom4,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,Shape_Area,-1,-1,zoom2\generalised_nrc_zoom2,Shape_Area,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,Shape_Area,-1,-1,zoom2\generalised_nrc_zoom3,Shape_Area,-1,-1,zoom2\generalised_nrc_zoom4,Shape_Area,-1,-1;NAME_0 "NAME_0" true true false 8000 Text 0 0,First,#,zoom2\generalised_nrc_zoom2,NAME_0,0,8000,zoom2\generalised_nrc_zoom3,NAME_0,0,8000,zoom2\generalised_nrc_zoom4,NAME_0,0,8000;country_zoom "country_zoom" true true false 4 Long 0 0,First,#,zoom2\generalised_nrc_zoom2,country_zoom,-1,-1,zoom2\generalised_nrc_zoom3,country_zoom,-1,-1,zoom2\generalised_nrc_zoom4,country_zoom,-1,-1;iso2 "iso2" true true false 8000 Text 0 0,First,#,zoom2\generalised_nrc_zoom2,iso2,0,8000,zoom2\generalised_nrc_zoom3,iso2,0,8000,zoom2\generalised_nrc_zoom4,iso2,0,8000', "NO_SOURCE_INFO")
arcpy.management.Merge(r"zoom1\generalised_nrc_zoom1;zoom1\World_Administrative_Divisio_zoom_1;zoom1\generalised_nrc_zoom2;zoom1\generalised_nrc_zoom3;zoom1\generalised_nrc_zoom4", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\zoom1_Merge", r'GID_0 "GID_0" true true false 8000 Text 0 0,First,#,zoom1\generalised_nrc_zoom1,GID_0,0,8000,zoom1\World_Administrative_Divisio_zoom_1,GID_0,0,8000,zoom1\generalised_nrc_zoom2,GID_0,0,8000,zoom1\generalised_nrc_zoom3,GID_0,0,8000,zoom1\generalised_nrc_zoom4,GID_0,0,8000;NAME_0 "NAME_0" true true false 8000 Text 0 0,First,#,zoom1\generalised_nrc_zoom1,NAME_0,0,8000,zoom1\generalised_nrc_zoom2,NAME_0,0,8000,zoom1\generalised_nrc_zoom3,NAME_0,0,8000,zoom1\generalised_nrc_zoom4,NAME_0,0,8000;country_size "country_size" true true false 4 Long 0 0,First,#,zoom1\generalised_nrc_zoom1,country_size,-1,-1,zoom1\World_Administrative_Divisio_zoom_1,country_size,-1,-1,zoom1\generalised_nrc_zoom2,country_size,-1,-1,zoom1\generalised_nrc_zoom3,country_size,-1,-1,zoom1\generalised_nrc_zoom4,country_size,-1,-1;country_zoom "country_zoom" true true false 4 Long 0 0,First,#,zoom1\generalised_nrc_zoom1,country_zoom,-1,-1,zoom1\generalised_nrc_zoom2,country_zoom,-1,-1,zoom1\generalised_nrc_zoom3,country_zoom,-1,-1,zoom1\generalised_nrc_zoom4,country_zoom,-1,-1;iso2 "iso2" true true false 8000 Text 0 0,First,#,zoom1\generalised_nrc_zoom1,iso2,0,8000,zoom1\generalised_nrc_zoom2,iso2,0,8000,zoom1\generalised_nrc_zoom3,iso2,0,8000,zoom1\generalised_nrc_zoom4,iso2,0,8000;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,zoom1\generalised_nrc_zoom1,Shape_Length,-1,-1,zoom1\World_Administrative_Divisio_zoom_1,Shape_Length,-1,-1,zoom1\generalised_nrc_zoom2,Shape_Length,-1,-1,zoom1\generalised_nrc_zoom3,Shape_Length,-1,-1,zoom1\generalised_nrc_zoom4,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,zoom1\generalised_nrc_zoom1,Shape_Area,-1,-1,zoom1\World_Administrative_Divisio_zoom_1,Shape_Area,-1,-1,zoom1\generalised_nrc_zoom2,Shape_Area,-1,-1,zoom1\generalised_nrc_zoom3,Shape_Area,-1,-1,zoom1\generalised_nrc_zoom4,Shape_Area,-1,-1;NAME "Name" true true false 44 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,NAME,0,44;COUNTRY "Country" true true false 50 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,COUNTRY,0,50;ISO_CODE "ISO Code" true true false 6 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,ISO_CODE,0,6;ISO_CC "ISO Country" true true false 2 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,ISO_CC,0,2;ISO_SUB "ISO Admin" true true false 5 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,ISO_SUB,0,5;ADMINTYPE "Admin Type" true true false 50 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,ADMINTYPE,0,50;DISPUTED "Disputed" true true false 4 Long 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,DISPUTED,-1,-1;NOTES "Notes" true true false 254 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,NOTES,0,254;AUTONOMOUS "Autonomous" true true false 4 Long 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,AUTONOMOUS,-1,-1;COUNTRYAFF "Affiliation" true true false 100 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,COUNTRYAFF,0,100;CONTINENT "Continent" true true false 13 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,CONTINENT,0,13;LAND_TYPE "Land_Type" true true false 20 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,LAND_TYPE,0,20;LAND_RANK "Land_Rank" true true false 4 Long 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,LAND_RANK,-1,-1', "NO_SOURCE_INFO")

## WDPA
Using non generalised polygons to extract the information
- population
- percentage protected (June version, MOL data)
- ecosystem majority